In [1]:
# Basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots
sns.set(rc={'figure.figsize':(11, 4)})

# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs


# settings
import warnings
warnings.filterwarnings("ignore")

# Customer scripts and methoods
import getPreProcessingFunction as PPM
import BiVEDA_Function as BiVEDA
import UniVEDA_catFunction as catUniVEDA
import UniVEDA_conti_methods as contiUniVEDA

getFuncs()
getFuncs()
getFuncs()
getFuncs()


In [229]:
def getResult_AdFuller_OR_kpss(_label_col,_df,testType=1):
#     print("""
#     for dickeyFuller -> testType = 0
#     for kpss -> testType = 1
#     for acf -> testType = 2
#     for pacf -> testType = 3
#     for visual and MA ->testType = 4
#     """)
    
    if testType == 1:
        from statsmodels.tsa.stattools import adfuller
        addfull=adfuller(_df[_label_col], autolag='AIC')
        print("\n\n > Is the data stationary via addfuller test?")
        print("Test statistic = {:.3f}".format(addfull[0]))
        print("P-value = {:.3f}".format(addfull[1]))
        print("#Lag Used: = {:.3f}".format(addfull[2]))
        print("Critical values :")
        for k, v in addfull[4].items():
            print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<addfull[0] else "", 100-int(k[:-1])))

        def isStationary(tstats):
            if addfull[0] < 0.5:
                return 'TS data is stationary'
            else:
                return 'TS data is non-stationary'    
        print(isStationary(addfull[0]))
    if testType == 0:
        from statsmodels.tsa.stattools import kpss
        print("\n\n > Is the data stationary via kpss test?")
        kpss_result=kpss(_df[_label_col],regression='c')
        print("Test statistic = {:.3f}".format(kpss_result[0]))
        print("P-value = {:.3f}".format(kpss_result[1]))
        print("#Lag Used: = {:.3f}".format(kpss_result[2]))
        print("Critical values :")
        for k, v in kpss_result[3].items():
            print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<kpss_result[0] else "", 100.0-float(k[:-1])))


        def isStationary(tstats):
            if kpss_result[0] < 0.5:
                return 'TS data is stationary'
            else:
                return 'TS data is non-stationary'    
        print(isStationary(kpss_result[0]))
    if testType == 2:
        from statsmodels.graphics.tsaplots import plot_acf
        plt.figure(figsize=(20,6))
        ax= plt.subplot(111)
        plot_acf(_df[_label_col],ax=ax)
        plt.xticks(fontsize=20)
        plt.title("AutoCorrelation plot",fontsize=30,color='grey')
        plt.yticks(fontsize=20)
        plt.xlabel("#No of lags",fontsize=20)
        plt.ylabel("correlation value -1<>1",fontsize=20)
    if testType == 3:
        from statsmodels.graphics.tsaplots import plot_pacf
        plt.figure(figsize=(20,6))
        ax= plt.subplot(111)
        plot_pacf(_df[_label_col],ax=ax)
        plt.xticks(fontsize=20)
        plt.title("Partial AutoCorrelation plot",fontsize=30,color='grey')
        plt.yticks(fontsize=20)
        plt.xlabel("#No of lags",fontsize=20)
        plt.ylabel("correlation value -1<>1",fontsize=20)
        
    if testType == 4:
        print("\n\n1. use ploting to test stationarity in dataset(moving Average)")
        plt.rc('xtick', labelsize=25)     
        plt.rc('ytick', labelsize=25)
        plt.figure(figsize=(26,10))
        plt.rc('legend',fontsize=20) # using a size in points

        plt.suptitle("Rolling average(Original hourly data) to test stationarity in data", y=1.0, fontsize=30)

        # 1. Original TS Junction 1
        plt.plot(_df[_label_col],label='Orig Train Count',color='grey')

        # 2. Original TS Junction 1 Rolling mean and std
        plt.plot(_df[_label_col].rolling(window=24).mean(),label='Orig Rolling mean',color='brown' )
        plt.plot(_df[_label_col].rolling(window=24).std(),label='Orig Rolling std',color='blue' )
        plt.legend(loc='best')

def ARIMAcorrPlot(_label_col,_df):
    from statsmodels.tsa.stattools import acf, pacf 
    lag_acf = acf(_df.dropna()[_label_col], nlags=30) 
    lag_pacf = pacf(_df.dropna()[_label_col], nlags=30, method='ols')
    lag_acf,lag_pacf

    # Lets plot Autocorrelation Function
    figure = plt.figure(figsize=(25,7))
    plt.rc('xtick', labelsize=25)     
    plt.rc('ytick', labelsize=25)
    plt.rc('legend',fontsize=20) # using a size in points
    plt.plot(lag_acf) 
    plt.axhline(y=0,linestyle='--',color='gray') 
    plt.axhline(y=-1.96/np.sqrt(len(_df.dropna())),linestyle='--',color='Red',label='Lower Confidence Interval') 
    plt.axhline(y=1.96/np.sqrt(len(_df.dropna())),linestyle='--',color='Blue',label='Upper Confidence Interval') 
    plt.title('Autocorrelation Function (Give Q value on first cut point Upper CI)',fontsize=35) 
    plt.legend(loc='best')


    # Lets plot Partial Autocorrelation Function
    figure = plt.figure(figsize=(25,7))
    plt.rc('xtick', labelsize=25)     
    plt.rc('ytick', labelsize=25)
    plt.rc('legend',fontsize=20) # using a size in points
    plt.plot(lag_pacf) 
    plt.axhline(y=0,linestyle='--',color='gray') 
    plt.axhline(y=-1.96/np.sqrt(len(_df.dropna())),linestyle='--',color='red',label='Lower Confidence Interval') 
    plt.axhline(y=1.96/np.sqrt(len(_df.dropna())),linestyle='--',color='blue',label='Upper Confidence Interval') 
    plt.title('Partial Autocorrelation Function (Give P value on first cut point Upper CI)',fontsize=35) 
    plt.legend(loc='best')

In [2]:
# Import all of them 
path="/Users/keeratjohar2305/Downloads/AV_ML_practice_Notebooks/JanataHACK_IOT_TS"

train=pd.read_csv(path+"/train_aWnotuB.csv")
sub=pd.read_csv(path+"/sample_submission_KVKNmI7.csv")
test=pd.read_csv(path+"/test_BdBKkAj_L87Nc3S.csv")

train_org=train.copy()
test_org=test.copy()

In [3]:
from datetime import datetime    # To access datetime 
from pandas import Series        # To work on series 
# reseting the index with datatime
train.DateTime= pd.to_datetime(train.DateTime,format='%Y-%m-%d %H:%M')
test.DateTime= pd.to_datetime(test.DateTime,format='%Y-%m-%d %H:%M')

#AP.Month= pd.to_datetime(AP.Month,format='%Y-%m')
#AP.index=AP.Month
train.index = train.DateTime
test.index = test.DateTime
# if 'ID' in train.columns:
#     train = train.drop('ID',axis=1)

    
df = train.copy()
df_test= test.copy()

In [4]:
label_col='Vehicles'

In [5]:
def applier(row):
    if row == 5 or row == 6:
        return 1
    else:
        return 0

train['year'] =train.DateTime.dt.year
train['day'] = train.DateTime.dt.day
train['month'] = train.DateTime.dt.month
train['Hour'] = train.DateTime.dt.hour
train['day of week'] = train['DateTime'].dt.dayofweek
train['weekend'] = train['DateTime'].dt.dayofweek.apply(applier)

def applier(row):
    if row == 5 or row == 6:
        return 1
    else:
        return 0

test['year'] =test.DateTime.dt.year
test['day'] = test.DateTime.dt.day
test['month'] = test.DateTime.dt.month
test['Hour'] = test.DateTime.dt.hour
test['day of week'] = test['DateTime'].dt.dayofweek
test['weekend'] = test['DateTime'].dt.dayofweek.apply(applier)

# SARIMA with LOG TRANSFORMATION
# best score : 7.24

In [180]:
def predictonJunction_dataWith_SARIMA_log(Jid=1):
    print ("\n \n --------  Junction "+ str(Jid) + " processing  --------- ")
    
    Junction_test=test[test['Junction']==Jid]
    Junction=train[train['Junction']==Jid]
    
    Junction['ratio']=Junction[label_col]/Junction[label_col].sum()
    Junction_temp=Junction.groupby(['Hour'])['ratio'].sum()
    Junction_temp=pd.DataFrame(Junction_temp).reset_index()

    
    Junction_resampled=Junction.resample('D').mean()
    Junction_resampled_test=Junction_test.resample('D').mean()
    
    
    from decimal import localcontext, Decimal, ROUND_HALF_UP
    Junction_resampled[label_col]=Junction_resampled[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
    Junction_resampled[label_col]=Junction_resampled[label_col].astype('int32')
    Junction_resampled['Junction']=Junction_resampled['Junction'].astype('int32')
    
    
#    print("\n \n---------------------------------- Original data Stationary test stats-------------------------------")
#     getResult_AdFuller_OR_kpss(label_col,Junction,0)
#     getResult_AdFuller_OR_kpss(label_col,Junction,1)
#     getResult_AdFuller_OR_kpss(label_col,Junction,4)
#     getResult_AdFuller_OR_kpss(label_col,Junction,2)
#     getResult_AdFuller_OR_kpss(label_col,Junction,3)
    
    
    
    # 2.2 Split the data into traing and validation dataset
    J_train=pd.DataFrame(Junction_resampled.ix['2015-11-01 00:00:00':'2017-03-31 23:59:59'],columns=[label_col])
    J_val=pd.DataFrame(Junction_resampled.ix['2017-03-31 23:59:59':],columns=[label_col])
    
    
    # 2.2 Log Transformation
    Junction_resampled_log=pd.DataFrame(np.log(Junction_resampled),columns=[label_col])
    J_log_train=pd.DataFrame(np.log(J_train),columns=[label_col])
    J_log_val=pd.DataFrame(np.log(J_val),columns=[label_col])
    #print(J_log_train.head())
    
    # SARIMAX Model building
    from sklearn.metrics import mean_squared_error
    import statsmodels.api as sm
    y_hat_avg = J_val.copy() 
    SARIMA_fit1 = sm.tsa.statespace.SARIMAX(J_log_train[label_col], order=(2, 1, 4),seasonal_order=(0,1,1,7)).fit() 
    y_hat_avg['SARIMA'] = np.exp(SARIMA_fit1.predict(start="2017-04-01", end="2017-06-30", dynamic=True))
    rms = np.sqrt(mean_squared_error(J_val[label_col], y_hat_avg.SARIMA)) 
    
    print("Validation RMSE :",rms)
#     plt.figure(figsize=(16,8)) 
#     plt.plot(J_train[label_col], label='Train') 
#     plt.plot(J_val[label_col], label='Valid') 
#     plt.plot(y_hat_avg['SARIMA'], label='SARIMA prediction') 
#     plt.legend(loc='best') 
#     plt.title("Sarima rmse score:" + str(rms),fontsize=35)
#     plt.show()

    
    

    # Make test prediction with SARIMA_fit1
    
    SARIMA_fit1 = sm.tsa.statespace.SARIMAX(Junction_resampled_log[label_col], order=(2, 1, 4),seasonal_order=(0,1,1,7)).fit() 
    
    
    predict=np.exp(SARIMA_fit1.predict(start="2017-07-01", end="2017-10-31", dynamic=True))
    Junction_resampled_test1=Junction_resampled_test.copy()
    Junction_resampled_test1['predict']=predict
    #print(Junction_resampled_test1.head(5))
    
    merge=pd.merge(Junction_test,Junction_resampled_test1, on=('day','month', 'year'), how='left')
    #print(merge.head(5))
    colDrop=['ID_y','Junction_y','day','month','year','Hour_y','day of week_y','weekend_y','weekend_x','day of week_x']
    merge.drop(colDrop,1,inplace=True)
    merge.columns=['DateTime','Junction','ID','Hour','predict']
    #print(merge.head(4))

    Junction_subFile=pd.merge(merge,Junction_temp,on='Hour',how='left')
    Junction_subFile[label_col]=Junction_subFile['predict']*Junction_subFile['ratio']*24
    Junction_subFile.drop(['Hour','ratio','predict','DateTime','Junction'],1,inplace=True)
    #Junction_subFile[label_col]=Junction_subFile[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
    #Junction_subFile[label_col]=Junction_subFile[label_col].astype('int32')
    
    return Junction_subFile
#predictonJunction_dataWith_SARIMA_log(4)

In [181]:
J1_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(1) 
J2_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(2) 
J3_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(3) 
J4_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(4) 

# Final Prediction
final_SARIMA_pred=J1_SARIMA_pred.append(J2_SARIMA_pred)
final_SARIMA_pred=final_SARIMA_pred.append(J3_SARIMA_pred)
final_SARIMA_pred=final_SARIMA_pred.append(J4_SARIMA_pred)
final_SARIMA_pred.shape,sub.shape
final_SARIMA_pred.head()
sub.head()
final_sub=pd.merge(sub,final_SARIMA_pred, on='ID',how='inner').drop('Vehicles_x',1)
final_sub.columns=sub.columns
final_sub.to_csv("AV_Junction_traffic.csv",index=False)
final_sub.shape


 
 --------  Junction 1 processing  --------- 
Validation RMSE : 4.662298438912607

 
 --------  Junction 2 processing  --------- 
Validation RMSE : 1.7364273994907642

 
 --------  Junction 3 processing  --------- 
Validation RMSE : 6.819970031209976

 
 --------  Junction 4 processing  --------- 
Validation RMSE : 2.101250832355623


(11808, 2)

In [182]:
final_sub.head(10)

,ID,Vehicles
0,20170701001,56.935221
1,20170701011,48.741659
2,20170701021,42.208512
3,20170701031,36.635580
4,20170701041,31.934826
5,20170701051,29.959117
6,20170701061,32.465094
7,20170701071,36.754327
8,20170701081,40.748740
9,20170701091,48.551254


# ARIMA PREDICTION 
# best score : 12.08

In [343]:
def predictonJunction_dataWith_ARIMA(Jid=1,arima_params=[1,1,1]):
    def check_prediction_log(_label_name,predict_log, given_set):
        #predict = np.exp(predict_log)
        figure = plt.figure(figsize=(25,8))
        plt.plot(given_set[_label_name], label = "Given set")
        plt.plot(predict_log[_label_name], color = 'red', label = "Predict")
        plt.legend(loc= 'best')
        plt.title('RMSE: %.4f'% (np.sqrt(np.dot(predict_log[_label_name], given_set[_label_name]))/given_set.shape[0]),fontsize=35)
        plt.show()
    
    def check_prediction_diff(_label_name,_predict_diff, given_set):
        predict_diff= _predict_diff.cumsum().shift().fillna(0)
        predict_base = pd.Series(np.ones(given_set.shape[0]) * np.log(given_set[_label_name])[0], index = given_set.index)
        predict_log = predict_base.add(predict_diff,fill_value=0)
        predict = pd.DataFrame(np.exp(predict_log),columns=[_label_name])
        
        print('RMSE: %.4f'% (np.sqrt(np.dot(predict[_label_name], given_set[_label_name]))/given_set.shape[0]))
        from decimal import localcontext, Decimal, ROUND_HALF_UP
        predict[label_col]=predict[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
        predict[label_col]=predict[label_col].astype('int32')
                                                                         
        return predict
    
    print ("\n \n --------  Junction "+ str(Jid) + " processing  --------- ")
    
    Junction_test=test[test['Junction']==Jid]
    Junction=train[train['Junction']==Jid]
    
    Junction['ratio']=Junction[label_col]/Junction[label_col].sum()
    Junction_temp=Junction.groupby(['Hour'])['ratio'].sum()
    Junction_temp=pd.DataFrame(Junction_temp).reset_index()

    
    Junction_resampled=Junction.resample('D').mean()
    Junction_resampled_test=Junction_test.resample('D').mean()
    
    from decimal import localcontext, Decimal, ROUND_HALF_UP
    Junction_resampled[label_col]=Junction_resampled[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
    Junction_resampled[label_col]=Junction_resampled[label_col].astype('int32')
    Junction_resampled['Junction']=Junction_resampled['Junction'].astype('int32')
    
    
    
#    print("\n \n---------------------------------- Original data Stationary test stats-------------------------------")
#     getResult_AdFuller_OR_kpss(label_col,Junction,0)
#     getResult_AdFuller_OR_kpss(label_col,Junction,1)
#     getResult_AdFuller_OR_kpss(label_col,Junction,4)
#     getResult_AdFuller_OR_kpss(label_col,Junction,2)
#     getResult_AdFuller_OR_kpss(label_col,Junction,3)
    
    
    # 2.2 Split the data into traing and validation dataset
    J_train=pd.DataFrame(Junction_resampled.ix['2015-11-01 00:00:00':'2017-03-31 23:59:59'],columns=[label_col])
    J_val=pd.DataFrame(Junction_resampled.ix['2017-03-31 23:59:59':],columns=[label_col])
    
    # create a differenced series
    def difference(dataset, interval=1):
        diff = list()
        for i in range(interval, len(dataset)):
            value = dataset[i] - dataset[i - interval]
            diff.append(value)
        return numpy.array(diff)

    
    
    # 2.2 Log Transformation
    Junction_resampled_log=pd.DataFrame(np.log(Junction_resampled),columns=[label_col])
    J_log_train=pd.DataFrame(np.log(J_train),columns=[label_col])
    J_log_val=pd.DataFrame(np.log(J_val),columns=[label_col])
    #print(J_log_train.head())
    
    # 2.3 differencing using moving average
    Junction_resampled_log_ma = Junction_resampled_log - Junction_resampled_log.rolling(60).mean()
    J1_train_log_ma = J_log_train - J_log_train.rolling(60).mean()
    J1_val_log_ma = J_log_val - J_log_val.rolling(60).mean()
    
    # 2.4 Differencing can help to make the series stable and eliminate the trend using shift
    Junction_resampled_withoutSeanality = Junction_resampled_log - Junction_resampled_log.shift(1) 
    J_train_withoutSeanality = J_log_train - J_log_train.shift() 
    J_val_withoutSeanality = J_log_val - J_log_val.shift() 
    
        
    print("\n \n---------------------------------- Original data Stationary test stats-------------------------------")
#     getResult_AdFuller_OR_kpss(label_col,J_train_withoutSeanality.dropna(),0)
#     getResult_AdFuller_OR_kpss(label_col,J_train_withoutSeanality.dropna(),1)
#     getResult_AdFuller_OR_kpss(label_col,J_train_withoutSeanality.dropna(),4)
#     getResult_AdFuller_OR_kpss(label_col,J_train_withoutSeanality.dropna(),2)
#     getResult_AdFuller_OR_kpss(label_col,J_train_withoutSeanality.dropna(),3)
#     ARIMAcorrPlot(label_col,J_train_withoutSeanality.dropna())
   
    
    

    # J1_log_val,J1_val_log_ma,J1_val_withoutSeanality
    # ARIMA MODEL
    model = ARIMA(J_log_train.dropna().dropna(), order=(arima_params[0], arima_params[1], arima_params[2]))  
    ARIMA_fit = model.fit(disp=-1)  
#     plt.figure(figsize=(18,10))
#     plt.plot(J_log_train[label_col],  label='original log',color='pink')  
#     plt.plot(ARIMA_fit.fittedvalues, color='red', label='prediction') 
#     plt.legend(loc='best') 
#     plt.show()
    
    # Validation Prediction
    ARIMA_validation_prediction=ARIMA_fit.predict(start="2017-04-01", end="2017-06-30")
    j_val_pred=check_prediction_diff(label_col,ARIMA_validation_prediction, J_val)
    #check_prediction_log(label_col,j_val_pred,J_val)
    #print(J_log_train.shape)
    #print(j_val_pred.head())
    
    
#     # ARIMAX Model building
    model = ARIMA(Junction_resampled_log, order=(2, 1, 2))  
    ARIMA_fit = model.fit(disp=-1) 
    
    from sklearn.metrics import mean_squared_error

    latest_val=np.exp(Junction_resampled_log.ix[Junction_resampled_log.index.max()][label_col])
    latest_val=np.exp(Junction_resampled_log.ix['2017-06-10':]).mean().values[0]
    print(latest_val)
#     # Make test prediction with ARIMA_fit1
    predict=ARIMA_fit.predict(start="2017-07-01", end="2017-10-31")
    predictions_ARIMA_diff_cumsum=predict.cumsum().shift().fillna(0) #, dynamic=True)
    predictions_ARIMA_log = pd.Series(latest_val,index=Junction_resampled_test.index)
    predict = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
#    print(predictions_ARIMA_log)
    Junction_resampled_test1=Junction_resampled_test.copy()
    Junction_resampled_test1['predict']=predict
#     #print(Junction_resampled_test1.head(5))
    
    merge=pd.merge(Junction_test,Junction_resampled_test1, on=('day','month', 'year'), how='left')
    #print(merge.head(5))
    colDrop=['ID_y','Junction_y','day','month','year','Hour_y','day of week_y','weekend_y','weekend_x','day of week_x']
    merge.drop(colDrop,1,inplace=True)
    merge.columns=['DateTime','Junction','ID','Hour','predict']
#     #print(merge.head(4))

    Junction_subFile=pd.merge(merge,Junction_temp,on='Hour',how='left')
    Junction_subFile[label_col]=Junction_subFile['predict']*Junction_subFile['ratio']*24
    Junction_subFile.drop(['Hour','ratio','predict','DateTime','Junction'],1,inplace=True)
# #     Junction_subFile[label_col]=Junction_subFile[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
# #     Junction_subFile[label_col]=Junction_subFile[label_col].astype('int32')
    return Junction_subFile
# Best tested params
#tt=predictonJunction_dataWith_ARIMA(1,arima_params=[2,1,2])
#tt=predictonJunction_dataWith_ARIMA(2,arima_params=[1,1,1])
#tt=predictonJunction_dataWith_ARIMA(3,arima_params=[1,1,1])
#tt=predictonJunction_dataWith_ARIMA(4,arima_params=[0,1,1])

In [344]:
J1_ARIMA_pred=predictonJunction_dataWith_ARIMA(1,arima_params=[2,1,2]) 
J2_ARIMA_pred=predictonJunction_dataWith_ARIMA(2,arima_params=[1,1,1]) 
J3_ARIMA_pred=predictonJunction_dataWith_ARIMA(3,arima_params=[1,1,1]) 
J4_ARIMA_pred=predictonJunction_dataWith_ARIMA(4,arima_params=[0,1,1]) 

# Final Prediction
final_ARIMA_pred=J1_ARIMA_pred.append(J2_ARIMA_pred)
final_ARIMA_pred=final_ARIMA_pred.append(J3_ARIMA_pred)
final_ARIMA_pred=final_ARIMA_pred.append(J4_ARIMA_pred)
final_ARIMA_pred.shape,sub.shape
final_ARIMA_pred.head()
sub.head()
final_sub=pd.merge(sub,final_ARIMA_pred, on='ID',how='inner').drop('Vehicles_x',1)
final_sub.columns=sub.columns
final_sub.to_csv("AV_Junction_traffic.csv",index=False)
final_sub.shape


 
 --------  Junction 1 processing  --------- 

 
---------------------------------- Original data Stationary test stats-------------------------------
RMSE: 6.4578
72.95238095238095

 
 --------  Junction 2 processing  --------- 

 
---------------------------------- Original data Stationary test stats-------------------------------
RMSE: 2.1556
25.666666666666668

 
 --------  Junction 3 processing  --------- 

 
---------------------------------- Original data Stationary test stats-------------------------------
RMSE: 1.9676
19.047619047619047

 
 --------  Junction 4 processing  --------- 

 
---------------------------------- Original data Stationary test stats-------------------------------
RMSE: 0.6879
8.619047619047619


(11808, 2)

# ARIMA and SARIMA Mix
# BEST SCORE : 7.35

In [342]:
J1_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(1) 
J2_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(2) 
J3_ARIMA_pred=predictonJunction_dataWith_ARIMA(3,arima_params=[1,1,1]) 
J4_ARIMA_pred=predictonJunction_dataWith_ARIMA(4,arima_params=[0,1,1]) 
J4_SARIMA_pred=predictonJunction_dataWith_SARIMA_log(4)
# Final Prediction
final_SARIMA_pred=J1_SARIMA_pred.append(J2_SARIMA_pred)
final_SARIMA_pred=final_SARIMA_pred.append(J3_ARIMA_pred)
final_SARIMA_pred=final_SARIMA_pred.append(J4_SARIMA_pred)
final_SARIMA_pred.shape,sub.shape
final_SARIMA_pred.head()
sub.head()
final_sub=pd.merge(sub,final_SARIMA_pred, on='ID',how='inner').drop('Vehicles_x',1)
final_sub.columns=sub.columns
final_sub.to_csv("AV_Junction_traffic.csv",index=False)
final_sub.shape


 
 --------  Junction 1 processing  --------- 
Validation RMSE : 4.662298438912607

 
 --------  Junction 2 processing  --------- 
Validation RMSE : 1.7364273994907642

 
 --------  Junction 3 processing  --------- 

 
---------------------------------- Original data Stationary test stats-------------------------------
RMSE: 1.9676
19.047619047619047

 
 --------  Junction 4 processing  --------- 

 
---------------------------------- Original data Stationary test stats-------------------------------
RMSE: 0.6879
8.619047619047619

 
 --------  Junction 4 processing  --------- 
Validation RMSE : 2.101250832355623


(11808, 2)

In [375]:
#train['DateTime'].dt.date.head()
pd.to_datetime(train['DateTime'].dt.date,format='%Y-%m-%d').dtypes

dtype('<M8[ns]')

In [389]:
def predictonJunction_XGB_regressor(Jid=1):
    import xgboost as xgb
    from xgboost import plot_importance, plot_tree
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    def check_prediction_log(_label_name,predict_log, given_set):
        #predict = np.exp(predict_log)
        figure = plt.figure(figsize=(25,8))
        plt.plot(given_set[_label_name], label = "Given set")
        plt.plot(predict_log[_label_name], color = 'red', label = "Predict")
        plt.legend(loc= 'best')
        plt.title('RMSE: %.4f'% (np.sqrt(np.dot(predict_log[_label_name], given_set[_label_name]))/given_set.shape[0]),fontsize=35)
        plt.show()
    
    def check_prediction_diff(_label_name,_predict_diff, given_set):
        predict_diff= _predict_diff.cumsum().shift().fillna(0)
        predict_base = pd.Series(np.ones(given_set.shape[0]) * np.log(given_set[_label_name])[0], index = given_set.index)
        predict_log = predict_base.add(predict_diff,fill_value=0)
        predict = pd.DataFrame(np.exp(predict_log),columns=[_label_name])
        
        print('RMSE: %.4f'% (np.sqrt(np.dot(predict[_label_name], given_set[_label_name]))/given_set.shape[0]))
        from decimal import localcontext, Decimal, ROUND_HALF_UP
        predict[label_col]=predict[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
        predict[label_col]=predict[label_col].astype('int32')
                                                                         
        return predict
    
    print ("\n \n --------  Junction "+ str(Jid) + " processing  --------- ")
    
    Junction_test=test[test['Junction']==Jid]
    Junction=train[train['Junction']==Jid]
    
    Junction1=pd.DataFrame(Junction[label_col].values,columns=[label_col],index=Junction.index)
    #Junction1['date'] = Junction['DateTime'].dt.date.astype('object').astype('int32')
    #Junction1['date'] = pd.to_datetime(Junction1['date'],format='%Y-%m-%d')
    Junction1['Hour'] = Junction['DateTime'].dt.hour
    Junction1['dayofweek'] = Junction['DateTime'].dt.dayofweek
    Junction1['quarter'] = Junction['DateTime'].dt.quarter
    Junction1['month'] = Junction['DateTime'].dt.month
    Junction1['year'] = Junction['DateTime'].dt.year
    Junction1['dayofyear'] = Junction['DateTime'].dt.dayofyear
    Junction1['dayofmonth'] = Junction['DateTime'].dt.day
    Junction1['weekofyear'] = Junction['DateTime'].dt.weekofyear
        
   
    # 2.2 Split the data into traing and validation dataset
    J_train=Junction1.ix['2015-11-01 00:00:00':'2017-03-31 23:59:59']
    J_val=Junction1.ix['2017-03-31 23:59:59':]
    
    xJ_train=J_train.drop(label_col,1)
    yJ_train=J_train[label_col]
    
    xJ_val=J_val.drop(label_col,1)
    yJ_val=J_val[label_col]
    
    print(xJ_train.dtypes)
    # 2.2 Log Transformation
#     J_log_train=J_train.copy()
#     J_log_val=J_val.copy()
#     J_log_train[label_col]=np.log(J_train[label_col])
#     J_log_val[label_col]=np.log(J_val[label_col])
#     print(J_log_train.head())
    
    reg = xgb.XGBRegressor(n_estimators=1000)
    reg.fit(xJ_train, yJ_train,
        eval_set=[(xJ_train, yJ_train), (xJ_val, yJ_val)],
        early_stopping_rounds=50,
      verbose=False) # Change verbose to True if you want to see it train
    xpred=reg.predict(xJ_train)
    print(np.sqrt(mean_squared_error(yJ_train,xpred)))
    
    vpred=reg.predict(xJ_val)
    print(np.sqrt(mean_squared_error(yJ_val,vpred)))
    
   # _ = plot_importance(reg, height=0.9)
#     merge=pd.merge(Junction_test,Junction_resampled_test1, on=('day','month', 'year'), how='left')
#     #print(merge.head(5))
#     colDrop=['ID_y','Junction_y','day','month','year','Hour_y','day of week_y','weekend_y','weekend_x','day of week_x']
#     merge.drop(colDrop,1,inplace=True)
#     merge.columns=['DateTime','Junction','ID','Hour','predict']
# #     #print(merge.head(4))

#     Junction_subFile=pd.merge(merge,Junction_temp,on='Hour',how='left')
#     Junction_subFile[label_col]=Junction_subFile['predict']*Junction_subFile['ratio']*24
#     Junction_subFile.drop(['Hour','ratio','predict','DateTime','Junction'],1,inplace=True)
# #     Junction_subFile[label_col]=Junction_subFile[label_col].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
# #     Junction_subFile[label_col]=Junction_subFile[label_col].astype('int32')
#    return Junction_subFile
# Best tested params
#tt=predictonJunction_dataWith_ARIMA(1,arima_params=[2,1,2])
#tt=predictonJunction_dataWith_ARIMA(2,arima_params=[1,1,1])
#tt=predictonJunction_dataWith_ARIMA(3,arima_params=[1,1,1])
#tt=predictonJunction_dataWith_ARIMA(4,arima_params=[0,1,1])
predictonJunction_XGB_regressor(1)

# UNDER CONSTRUCTION


 
 --------  Junction 1 processing  --------- 
Hour          int64
dayofweek     int64
quarter       int64
month         int64
year          int64
dayofyear     int64
dayofmonth    int64
weekofyear    int64
dtype: object
3.921020245793894
10.153192904572895
